## Encoding the knowledge base into Hyperdimensional Vectors

In this notebook the functions from the 'HDComputing' notebook are used to encode the McRae dataset. The following functions create an heteroassociative memory in which a knowledge base of Semantic Features representation of concepts is stored.

### Importing libraries and HD computing functions

In [3]:
import pandas as pd
import nltk

#Only done once... 
#nltk.download('wordnet')
#nltk.download('wordnet_ic')

from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic

# Declare functions and class...
%run HDComputing_basics.ipynb 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jobqu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\jobqu\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet_ic.zip.


### Functions for reading dataset

In [2]:
def TranslateFeats(ListFeat):
    "It receives a list of features such as ['is_blue', 'is_rectangular'] and it returns: [['color','blue'], ['shape','rectangular']"
    # Dataframe for excel document
    df = pd.read_excel(pathh + 'FEATS_brm.xlsx') #../McRaedataset/FEATS_brm.xlsx')
    ListPairs = []
    for feat in ListFeat:
        # Row for feature...
        row = df.loc[df['Feature'] == feat]       
        # Look for values in vec_feat and vec_value
        ListPairs.append([str(row['feat_name'].tolist()[0]), str(row['feat_value'].tolist()[0])])       
    return ListPairs

def ReadDefinitions():
    "Given an xlsx file it returns all the concepts feature values as they appear in the original dataset"
    #Dataframe for excel document
    df = pd.read_excel( pathh + 'CONCS_FEATS_concstats_brm.xlsx') #../McRaeDataset/CONCS_FEATS_concstats_brm.xlsx') #MINI_
    #Create a list with all concept names
    names = set(df['Concept'])
    # Extract list of features for each name
    Concepts = []
    for n in names:
        row = df.loc[df['Concept'] == n]
        Concepts.append([str(n), map(str,list(row['Feature']))])
    return Concepts

def ClosestConcepts (concept, nc):
    "Given a concept label this function reads the distance matrix from McRae's and returns the 'nc' closests concepts in a list"
    # Excel document to data frame...
    try:
        df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','1st_200') #../McRaeDataset/cos_matrix_brm_IFR.xlsx', '1st_200')
        ordered = df.sort_values(by=concept, ascending=False)[['CONCEPT', concept]]
    except: 
        try:
            df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','2nd_200') # ('../McRaeDataset/cos_matrix_brm_IFR.xlsx', '2nd_200')
            ordered = df.sort_values(by=concept, ascending=False)[['CONCEPT', concept]]
        except:
            df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','last_141') #('../McRaeDataset/cos_matrix_brm_IFR.xlsx', 'last_141')
            ordered = df.sort_values(by=concept, ascending=False)[['CONCEPT', concept]]
    
    L1 = list(ordered['CONCEPT'][0:nc])
    L1 = map(str, L1)
    L2 = zip(L1,list(ordered[concept][0:nc]))
    L2 = map(list, L2)
    
    return L2

## Storing ID vectors into memory

### Creating definitions dictionary

In [3]:
def CreateDictionary():
    global Dict_defs
    data = ReadDefinitions()
    for concept in data:
        Dict_defs[concept[0]] = TranslateFeats(concept[1])

### Memory functions

In [4]:
def flat_list (L):
    "Recursive function that flats a list of lists (at any level)"
    if L == []:
        return L
    if type(L[0]) is list:
        return flat_list(L[0]) + flat_list(L[1:])
    return L[:1] + flat_list(L[1:])

def FeatureVectors(Dic):
    "It extract from the definition dictionary all the feature type vectors ('is','has','color', etc...)"
    global feature_vectors
    featt = []
    vals = Dic.values()
    for l in vals:
        for p in l:
            featt.append(p[0])
    feature_vectors = list(set(featt))
    
    
def SaveConcepts(Dic):
    """Given a definitions dictionary it stores in memory the entire set of concepts in the dictionary (including feature vectors)"""
    keys = Dic.keys()
    vals = list( Dic.values() )
    all_concepts = list(set(flat_list(vals) + keys))
    # Process for storing list of concepts in memory
    for concept in all_concepts:
        HDvector(N,concept) #This creates an object and store it in memory    
    
def CreateSemanticPointer (PairList):
    "Turns list as [[feat1,feat_val],[feat2,feat_val],[feat3,feat_val]] into vector feat1*feat_val + feat2*feat_val ..."
    vecs = []
    for pair in PairList:
        vecs.append(Dict[pair[0]] * Dict[pair[1]])
    return ADD(vecs)

def SaveDefinitions(Dic):
    """Given the definitions dictionary, and having all its concepts previously stored in memory, this functions
       creates a definition vector (semantic pointer) using HD operations and assign it as a pointer to an 
       object vector (ID vector)."""
    global feature_vectors
    # Going through all elements in dictionary
    for key, value in Dic.iteritems():
        Dict[key].setPointer(CreateSemanticPointer(value))
        
def NormalizeHammDist (Dist_list):
    "Given a distance list of the form [['name', dist], ['name', dist], ... ], it normalize each distance and return a list with the same form"
    for i in range(len(Dist_list)):
        Dist_list[i][1] = round( 1. - Dist_list[i][1] / float(N), 3 ) 
    return Dist_list

### Initializing memory

In [5]:
def Init_mem():
    init()
    print("Begining to encode dataset...")
    thr = 0.45 * N
    # Read dataset and create definition dictionary
    CreateDictionary()
    # Feature vectors
    FeatureVectors(Dict_defs)
    # Save concepts into memory (ID vectors)
    SaveConcepts(Dict_defs)
    # Associate definitions to concepts into memory (SP vectors)
    SaveDefinitions(Dict_defs)
    print("End of encoding")

## Reading similarity from distance matrix (McRae)

In [6]:
def McRae_simi (pair_concepts):
    "Given a pair of concepts (in a list) it consults the similarity from the cos_matrix... file"
    try: 
        df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','1st_200')
        return list(df.loc[df['CONCEPT'] == pair_concepts[0]][pair_concepts[1]])[0]
    except:
        try:
            df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','2nd_200')
            return list(df.loc[df['CONCEPT'] == pair_concepts[0]][pair_concepts[1]])[0]
        except:
            df = pd.read_excel(pathh + 'cos_matrix_brm_IFR.xlsx','last_141')
            return list(df.loc[df['CONCEPT'] == pair_concepts[0]][pair_concepts[1]])[0]

## Semantic similarity using NLTK library functions
### Auxiliar functions

In [7]:
brown_ic = wordnet_ic.ic('ic-brown.dat')

def get_concepts_list ():
    "Returns a list of strings: the names of the concepts"
    df = pd.read_excel(pathh + 'CONCS_Synset_brm.xlsx') #../McRaedataset/CONCS_Synset_brm.xlsx')
    return map(str, list(df['Concept']))
    
def get_synset (concept):
    "Given a concept name (string) it returns its synset (string)"
    # Dataframe for excel document
    df = pd.read_excel(pathh + 'CONCS_Synset_brm.xlsx') #../McRaedataset/CONCS_Synset_brm.xlsx')
    row = df.loc[df['Concept'] == concept]
    return str(list(row['Synset'] )[0])

def apply_sim_metric ( similarity_metric, num, in_concept, corpus = None):
    "Given a similarity_metric function it returns a list of the num closest concepts to 'concept'"
    dist_list = []
    for c in Concepts:
        c_synset = wn.synset( get_synset(c) )
        if corpus:
            dist_list.append([c, round(similarity_metric(in_concept, c_synset, corpus), 3) ])
        else:
            dist_list.append([c, round(similarity_metric(in_concept, c_synset), 3) ])
    return sorted(dist_list, key = lambda r : r[1], reverse = True ) [:num]

def similarity_fun ( similarity_metric, pair, corpus = None):
    "Given a similarity_metric function it returns a list of the num closest concepts to 'concept'"
    c_synset_1 = wn.synset( get_synset(pair[0]))
    c_synset_2 = wn.synset( get_synset(pair[1]))
    if corpus:
        return round(similarity_metric(c_synset_1, c_synset_2, corpus), 3)
    else:
        return round(similarity_metric(c_synset_1, c_synset_2), 3)

## New similarity metric: Extended Gloss Overlap

In [8]:
import re
from nltk.stem.porter import PorterStemmer

HYPE = 'hypernymy'
HYPO = 'hyponymy'
GLOSS = 'gloss'
MERONYMY = 'meronymy'

def glossOverlap(gloss1, gloss2):
    # stopws = stopwords.words('english')
    stopws = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
    'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 
    'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 
    'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 
    'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 
    'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 
    'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 
    'in', 'out', 'on', 'off', 'over', 'under', 'then',  
    'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each',
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 
    'than', 's', 't', 'just', 'don']

    stemmer = PorterStemmer()

    # print "-"*50
    # for x in stopws:
    #  if stemmer.stem_word(x) != x:
    #     print x, stemmer.stem_word(x)

    def longestOverlap(a, b):
        now = [0]*len(b)
        bestOverlap = 0
        aStart = 0
        bStart = 0

        nextNonStopWord = [-1]*(len(a)+1)
        for i in range(len(a)-1, 0, -1):
            if a[i] not in stopws:
                nextNonStopWord[i] = i
            else:
                nextNonStopWord[i] = nextNonStopWord[i+1]

        for i in range(1, len(a)):
            prev = now
            now = [0]*len(b)
            if a[i] == '#':
                continue
            for j in range(1, len(b)):
                if b[j] == '#':
                    continue
                if a[i] == b[j]:
                    now[j] = max(now[j], prev[j-1] + 1)
                    if a[i] in stopws:
                        continue

                    overlap = now[j]
                    start = i - overlap + 1
                    start = nextNonStopWord[start]
                    overlap = i - start + 1
                    if bestOverlap < overlap:
                        bestOverlap = overlap
                        aStart = i - overlap + 1
                        bStart = j - overlap + 1

        return (bestOverlap, aStart, bStart)


    regex = ',|\.|\s|\?|\'|\"|!|;|-'
    #maybe check what happens if we don't stem the glosses
    a1 = ['#'] + [stemmer.stem(x.lower()) for x in re.split(regex, gloss1) if x]
    a2 = ['#'] + [stemmer.stem(x.lower()) for x in re.split(regex, gloss2) if x]

    score = 0
    (overlap, start1, start2) = longestOverlap(a1, a2)
    while overlap > 0:
        # print overlap
        # print a1[start1:start1+overlap]
        # print a2[start2:start2+overlap]
        a1[start1:start1+overlap] = ['#']
        a2[start2:start2+overlap] = ['#']
        score += overlap**2
        (overlap, start1, start2) = longestOverlap(a1, a2)

    return score


def getRelationGloss(synset, r):
    synsets = []
    if r == HYPE:
        synsets = synset.hypernyms()
    elif r == HYPO:
        synsets = synset.hyponyms()
    elif r == GLOSS:
        synsets = [synset]
    elif r == MERONYMY:
        synsets = synset.member_meronyms() + synset.part_meronyms() + synset.substance_meronyms()
    else:
        #to do: add other relations if necessary
        raise Exception("Unknown relation %s" % r)

    glosses = [x.definition() for x in synsets]
    strr = " ".join(glosses)
    return strr

def GlossRelatedness(synset1, synset2):
    relpairs = [(GLOSS, GLOSS)] #, (HYPE, HYPE), (HYPO, HYPO), (HYPE, GLOSS), (GLOSS, HYPE)]
    ret = 0
    for (r1, r2) in relpairs:
        gloss1 = getRelationGloss(synset1, r1)
        gloss2 = getRelationGloss(synset2, r2)

        ret += glossOverlap(gloss1, gloss2)
    return ret
